In [1]:
%matplotlib inline

# TensorFlow 全相连网络(Fully Connected Network)

In [2]:
# -*- coding: utf-8 -*-

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from six.moves import xrange

导入MNIST数据集<br>
[MNIST](http://yann.lecun.com/exdb/mnist/)(读作minist)，是由LeCunn等三位大佬在98年贡献的手写数字数据集。<br>
每张图片为28*28像素的二值图，算得上是深度学习的入门资料。地位如同helloworld程序。

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('./MNIST_data', one_hot=True)
print(
        '训练集大小: %s;\n验证集大小: %s;\n测试集大小: %s.' % \
        (mnist.train.num_examples, mnist.validation.num_examples, mnist.test.num_examples)
)
image_size = mnist.train.images[0].shape[0]
print('输入的大小：%s' % (image_size))


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
训练集大小: 55000;
验证集大小: 5000;
测试集大小: 10000.
输入的大小：784


数据展示

In [4]:
import numpy as np
from PIL import Image

Image.fromarray(np.reshape(a=mnist.train.images[0], newshape=(28, 28)), 'I')


## 正则化实现(regularization)

### i).自定义网络参数矩阵实现<br>
``
weight_a = tf.Variable(tf.random_normal(shape=[original, target], mean=.0, stddev=.1))
weight_b = tf.Variable(tf.zeros(shape=[target]))
tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(1e-4)(weight_a))
tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(1e-4)(weight_b))
tf.add_to_collection('losses', loss_function)
loss_op = tf.add_n(tf.get_collection('losses'))
``<br>

### ii).TensorFlow layer函数实现<br>
``
tf.layers.dense(kernel_regularizer=1e-4, bias_regularizer=1e-4)
tf.add_to_collection(name=tf.GraphKeys.REGULARIZATION_LOSSES, value=loss_function))
loss_op = tf.reduce_mean(tf.add_n(tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)))
``

## batch norm（一维）

### 方式1
``
mean, var = tf.nn.moments(input_tensor, axes=[0])
shift = tf.Variable(tf.zeros(input_tensor.get_shape()[-1]))
scale = tf.Variable(tf.zeros(input_tensor.get_shape()[-1]))
variance_epsilon = 1e-4
output = tf.nn.batch_normalization(input_data, mean, var, shift, scale, epsilon)
``

### 方式2
``
output = (input_tensor - mean) / tf.sqrt(var + variance_epsilon)
output = output * scale + offset
``

### 方式3
``
def mean_var_with_update():
    ema_apply_op = ema.apply([mean, var])
    with tf.control_dependencies([ema_apply_op]):
        return tf.identity(mean), tf.identity(var)
``<br>
``
ema = tf.train.ExponentialMovingAverage(decay=0.9)
mean, var = mean_var_with_update()
output = tf.nn.batch_normalization(input_tensor, mean, var, offset, scale, variance_epsilon)
``

## batch norm（二维）
``
mean, var = tf.nn.moments(input_tensor, [0, 1, 2], keep_dims=True)
shift = tf.Variable(tf.zeros(input_tensor.get_shape()[-1]))
scale = tf.Variable(tf.zeros(input_tensor.get_shape()[-1]))
epsilon = 1e-3
output = tf.nn.batch_normalization(input_data, mean, var, shift, scale, epsilon)
``

In [5]:
import tensorflow as tf

# 超参数(hyper-parameter)是指构成网络的参数，迭代次数/batch大小也可以作为超参数的一部分
training_epoch = 100
num_class = 10
learning_rate = 1e-3
batch_size = 100

graph = tf.Graph()
with graph.as_default():
    with tf.name_scope('placehold'):
        # placehold - define variable
        x = tf.placeholder(dtype=tf.float32, shape=[None, image_size], name='x')
        y = tf.placeholder(dtype=tf.float32, shape=[None, num_class], name='y')


    def linear(input_data, name=None, target=None, activation=tf.nn.relu, norm=False):
        original = int(list(input_data.shape)[-1])
        with tf.name_scope('layer_%s' % (name)):
            # define two variable for y=a*x+b
            weight_a = tf.Variable(tf.random_normal(shape=[original, target], mean=.0, stddev=.1),
                                   name='layer_%s_a' % (name))
            weight_b = tf.Variable(tf.zeros(shape=[target]), name='layer_%s_b' % (name))
            
            output = tf.matmul(input_data, weight_a) + weight_b
            # batch normalize
            if norm:
                mean, var = tf.nn.moments(output, axes=[0])
                offset = tf.Variable(tf.ones([target]))
                scale = tf.Variable(tf.ones([target]))
                variance_epsilon = 1e-4
                output = tf.nn.batch_normalization(output, mean, var, offset, scale, variance_epsilon)
                
        return activation(output)


    # MLP
    fc1 = linear(input_data=x, name='1', target=256)
    # dropout是神经网络中重要的提示网络性能的手段。它不是网络。
    # dropout可以放置norm过拟合，提升网络准确率，得到数据特征等作用。
    # 类似的还有batchnorm等，都对网络性能的提升有着关键性的作用
    fc1 = tf.layers.dropout(inputs=fc1, rate=0.7)
    fc2 = linear(input_data=fc1, name='2', target=64)
    fc3 = linear(input_data=fc2, name='3', target=num_class, activation=tf.nn.softmax)
    y_ = fc3

    with tf.name_scope('loss_op'):
        tf.add_to_collection('losses', tf.losses.mean_squared_error(y, y_))
        loss_op = tf.add_n(tf.get_collection('losses'))
    with tf.name_scope('train_op'):
        train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_op)
    with tf.name_scope('accuracy_op'):
        accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1)), tf.float32))


batch不宜太大

In [6]:
with tf.Session(graph=graph) as session:
    session.run(tf.global_variables_initializer())

    batch_episode = len(mnist.train.images) // batch_size
    for i in xrange(1, 1 + training_epoch):
        for _ in xrange(batch_episode):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            feed = {x: batch_x, y: batch_y}
            _, batch_cost, batch_acc = session.run([train_op, loss_op, accuracy], feed_dict=feed)
        if i % 10 == 0:
            loss_val, acc_val = session.run([loss_op, accuracy], feed_dict={
                x: mnist.test.images,
                y: mnist.test.labels
            })
            print('Epoch %s / %s, loss: %s, accuracy: %s' % (i, training_epoch, loss_val, acc_val))


Epoch 10 / 100, loss: 0.007501135, accuracy: 0.9764
Epoch 20 / 100, loss: 0.0076487865, accuracy: 0.9757
Epoch 30 / 100, loss: 0.00798492, accuracy: 0.9763
Epoch 40 / 100, loss: 0.006740521, accuracy: 0.9797
Epoch 50 / 100, loss: 0.0077452576, accuracy: 0.9778
Epoch 60 / 100, loss: 0.0072739497, accuracy: 0.9789
Epoch 70 / 100, loss: 0.007299751, accuracy: 0.9794
Epoch 80 / 100, loss: 0.0078655705, accuracy: 0.9785
Epoch 90 / 100, loss: 0.0074387654, accuracy: 0.9799
Epoch 100 / 100, loss: 0.006868942, accuracy: 0.9812


检验
estimaton

In [7]:
# 在深度学习中实际值/真实值叫做ground truth
Image.fromarray(np.reshape(a=mnist.test.images[0], newshape=(28, 28)), 'I')


In [8]:
with tf.Session(graph=graph) as session:
    session.run(tf.global_variables_initializer())

    predict = session.run([y_], feed_dict={
        x: mnist.test.images[0][None, ...],
        y: mnist.test.labels[0][None, ...]
    })
    # prediction
    print('predicted image is %s' % (np.argmax(a=predict)))


predicted image is 4


推荐资料：<br>
[Stanford UFLDL课程，神经网络](http://deeplearning.stanford.edu/wiki/index.php/Neural_Networks)<br>
[Stanford UFLDL课程，backprogation算法](http://deeplearning.stanford.edu/wiki/index.php/Backpropagation_Algorithm)<br>
[Stanford UFLDL课程，backprogation算法](http://deeplearning.stanford.edu/wiki/index.php/Backpropagation_Algorithm)<br>
[Stanford UFLDL课程，梯度检验与高级优化](http://deeplearning.stanford.edu/wiki/index.php/Gradient_checking_and_advanced_optimization)<br>
[Micheal Nielsen, *Neural Networks and Deep Learning*](http://neuralnetworksanddeeplearning.com/index.html)<br>
[sklearn库，神经网络](http://scikit-learn.org/stable/modules/neural_networks_supervised.html)<br>
[TensorFLow全相连网络实现MNIST识别（教程代码的版本太低）](http://www.tensorfly.cn/tfdoc/tutorials/mnist_beginners.html)<br>
